<a href="https://colab.research.google.com/github/tarjerw/TDT4173-project-group6/blob/main/Machine_Learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Political party analysis program that parses the tweets fetched from Twitter using Python

In [ ]:
###### IMPORT THE LIBRARIES  #########
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Candidate twitter accounts
Rep2012 = ["MittRomney","RickSantorum","RonPaul","newtgingrich","BuddyRoemer","SecretaryPerry","MicheleBachmann"]
Rep2016 = ["realDonaldTrump","tedcruz","marcorubio","JohnKasich","SecretaryCarson","JebBush","RandPaul","GovMikeHuckabee","CarlyFiorina","GovChristie","gov_gilmore"]
Rep2020 = ["realDonaldTrump"]
Dem2012 = ["BarackObama"]
Dem2016 =  ["MartinOMalley","BernieSanders","HillaryClinton"]
Dem2020 = ["JoeBiden","AndrewYang","BernieSanders","TulsiGabbard","ewarren","MikeBloomberg","amyklobuchar","PeteButtigieg","TomSteyer","DevalPatrick","MichaelBennet"]

##Retrieve data

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
#retrieves data
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/18a7-8eM-rcraKagnXPpCmYbC4uwWwBVq_iF_IwozYos/edit#gid=787454996')
sheet = wb.worksheet('BarackObama')
data = sheet.get_all_values()
df_all = pd.DataFrame(data)
df_all.columns = df_all.iloc[0]
df_all = df_all.iloc[1:]

In [ ]:
#shuffles data
df_all = df_all.sample(frac = 1)
df_all.iloc[20:30]

,Candidate,Tweet,Year,Party
8656,DonaldTrump,Thank you Mr. &amp; Mrs. @TomBarrackJr for the...,2016,Republican
8887,DonaldTrump,Thank you Fort Wayne Indiana!#Trump2016 #INPri...,2016,Republican
6547,BernieSanders,Wisconsin — don't know where to vote for Berni...,2016,Democrat
7881,DonaldTrump,RT @Avik: So @YouTube just took down a June 23...,2020,Republican
7037,DonaldTrump,The GREAT Bobby Bowden one of the best coaches...,2020,Republican
7266,DonaldTrump,HAPPY COLUMBUS DAY TO ALL!,2020,Republican
4973,HillaryClinton,Our teachers deserve more than just a pat on t...,2016,Democrat
8274,DonaldTrump,Congratulations to Dustin Johnson @DJohnsonPGA...,2020,Republican
2429,HillaryClinton,"""You don't grade the presidency on a curve."" —...",2016,Democrat
9619,DonaldTrump,Remember Cruz and Bush gave us Roberts who uph...,2016,Republican


In [ ]:
#shows the number of duplicated tweets
df_all = df_all.drop_duplicates() #removes all duplicates
df_all.duplicated().value_counts()

False    9782
dtype: int64

In [ ]:
#counts the number of democrate tweets and republican tweets
df_all.Party.value_counts()

Democrat      6761
Republican    3021
Name: Party, dtype: int64

##Sentiment Analysis using Bag-of-Words & N-Gram

In [ ]:
#import needed libraries
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

###Most common words

####Most common republican words



In [ ]:
data_rep = df_all.loc[df_all['Party'] == 'Republican']

rep_BagOfWords_vectorizer = CountVectorizer(max_df=0.35)
data_rep_BagOfWords = rep_BagOfWords_vectorizer.fit_transform(data_rep.Tweet)

freqs = zip(rep_BagOfWords_vectorizer.get_feature_names(), data_rep_BagOfWords.sum(axis=0).tolist()[0])    
freqs_list = list(freqs)
sorted_freps = sorted(freqs_list, key=lambda x: -x[1])
for i in range(30):
  print(sorted_freps[i+100])

('them', 84)
('bad', 83)
('because', 82)
('years', 82)
('him', 81)
('realdonaldtrump', 81)
('way', 81)
('crooked', 80)
('endorsement', 80)
('tonight', 79)
('said', 78)
('her', 77)
('carolina', 76)
('pennsylvania', 76)
('state', 76)
('time', 76)
('were', 75)
('china', 74)
('enjoy', 74)
('how', 74)
('there', 74)
('want', 74)
('against', 73)
('left', 73)
('sleepy', 71)
('florida', 70)
('day', 69)
('don', 69)
('down', 69)
('go', 67)


####Most common democrat words

In [ ]:
data_dem = df_all.loc[df_all['Party'] == 'Democrat']

dem_BagOfWords_vectorizer = CountVectorizer(max_df=0.35)
data_dem_BagOfWords = dem_BagOfWords_vectorizer.fit_transform(data_dem.Tweet)

freqs_dem = zip(dem_BagOfWords_vectorizer.get_feature_names(), data_dem_BagOfWords.sum(axis=0).tolist()[0])    
freqs_list = list(freqs_dem)
sorted_freps = sorted(freqs_list, key=lambda x: -x[1])
for i in range(30):
  print(sorted_freps[i+100])

('berniesanders', 200)
('want', 200)
('twitter', 199)
('would', 196)
('been', 194)
('crisis', 193)
('she', 193)
('go', 191)
('plan', 190)
('never', 189)
('better', 188)
('change', 185)
('economy', 177)
('amp', 176)
('clinton', 173)
('fight', 173)
('because', 170)
('first', 170)
('world', 170)
('live', 169)
('last', 168)
('covid', 164)
('election', 161)
('millions', 159)
('too', 159)
('only', 158)
('everyone', 156)
('join', 156)
('19', 155)
('million', 155)


###Splitting data

In [ ]:
# Splits dataset into Training data and Test data
X_train, X_test, y_train, y_test = train_test_split(
    df_all.Tweet, df_all.Party, test_size=0.3)

print("Democratic republican ratio in Training data:\n"+str(y_train.value_counts())+"\n")
print("Democratic republican ratio in Test data:\n"+str(y_test.value_counts()))

Democratic republican ratio in Training data:
Democrat      4725
Republican    2122
Name: Party, dtype: int64

Democratic republican ratio in Test data:
Democrat      2036
Republican     899
Name: Party, dtype: int64


####Creating Bag-of-Words Model

In [ ]:
# Create Bag of words
BagOfWords_vectorizer = CountVectorizer(max_df=0.35)
X_train_BagOfWords = BagOfWords_vectorizer.fit_transform(X_train)
X_test_BagOfWords = BagOfWords_vectorizer.transform(X_test)

####Creating N-Gram Model 

In [ ]:
# Create Bag of words
N_Gram_vectorizer = CountVectorizer(ngram_range=(1,3), max_df=0.35) #max_df=0.8 removes all words that are present more than 80% of the time
X_train_N_Gram = N_Gram_vectorizer.fit_transform(X_train)
X_test_N_Gram = N_Gram_vectorizer.transform(X_test)

In [ ]:
######### MOST COMMON WORDS IN ENTIRE DATASET ###########################
matrix = N_Gram_vectorizer.fit_transform(X_train)
freqs = zip(N_Gram_vectorizer.get_feature_names(), matrix.sum(axis=0).tolist()[0])    
# sort from largest to smallest
print("most frequent words and word combinations in N-Gram:\n", sorted(freqs, key=lambda x: -x[1]))

######### SEE SPECIFIC WORDS AND THEIR RESPECTIVE ID ##################

#BagOfWords_vectorizer.vocabulary_
#N_Gram_vectorizer.vocabulary_ # uncomment to compare



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



###Performing cross validation

A key challenge with machine learning, is that we can’t know how well our model will perform on new data until we actually test it.

To address this, we can split our initial dataset into separate training and test subsets.

There are different types of Cross Validation Techniques but the overall concept remains the same:

*   To partition the data into a number of subsets (given as argument for cv in cross_val_score below)
*   Hold out a set at a time and train the model on remaining set
*   Test model on hold out set
*   Repeat the process for each subset of the dataset




A cross validation is done in the code segment below


In [ ]:
# Cross-validation is a statistical method used to estimate the skill of machine learning models.
scores_BagOfWords = cross_val_score(LogisticRegression(), X_train_BagOfWords, y_train, cv=5)
scores_N_Gram = cross_val_score(LogisticRegression(), X_train_N_Gram, y_train, cv=5)

# Returns an array of scores of the estimator for each run of the cross validation.
print("Bag of Words, cross validation scores:",scores_BagOfWords) 
print("N-Gram, cross validation scores:",scores_N_Gram) 

Bag of Words, cross validation scores: [0.94160584 0.9379562  0.93352812 0.94010226 0.93498904]
N-Gram, cross validation scores: [0.94671533 0.9379562  0.93791088 0.93791088 0.93791088]


###Data Prediction

How well does the training data predict the test data?



Bag of Words

In [ ]:
# How well does the Bag of Words training data predict the test data
logistic_reg_BagOfWords = LogisticRegression()
logistic_reg_BagOfWords.fit(X_train_BagOfWords, y_train) #Fits the model according to the given training data.
print("The logistic regression, \ncreated and trained on Training data predicts", round(logistic_reg_BagOfWords.score(X_train_BagOfWords, y_train)*100,3), "% og the Training dataset correctly.")
print("More importantly, it predicts", round(logistic_reg_BagOfWords.score(X_test_BagOfWords, y_test)*100,3), "% og the Test dataset correctly!!!")

The logistic regression, 
created and trained on Training data predicts 99.576 % og the Training dataset correctly.
More importantly, it predicts 94.48 % og the Test dataset correctly!!!


N-Gram

In [ ]:
# How well does the N-Gram training data predict the test data
logistic_reg_N_Gram = LogisticRegression()
logistic_reg_N_Gram.fit(X_train_N_Gram, y_train) #Fits the model according to the given training data.
print("The logistic regression, \ncreated and trained on Training data predicts", round(logistic_reg_N_Gram.score(X_train_N_Gram, y_train)*100,3), "% og the Training dataset correctly.")
print("More importantly, it predicts", round(logistic_reg_N_Gram.score(X_test_N_Gram, y_test)*100,3), "% og the Test dataset correctly!!!")

The logistic regression, 
created and trained on Training data predicts 99.971 % og the Training dataset correctly.
More importantly, it predicts 95.537 % og the Test dataset correctly!!!


Logistic Regression is used when the dependent variable (target) is categorical.
Above, we are using a Binary Logistic Regression.
The categorical response has only two 2 possible outcomes: Democrate or Republican.

###Confusion Matrix

Compute confusion matrix to evaluate the accuracy of a classification.

By definition a confusion matrix C is such that Cij is equal to the number of observations known to be in group i and predicted to be in group j.

Thus in our binary classification, the count of true Republican tweets is C[0][0] and false Republican tweets is C[1][0]. Likewise, the count of true Democrat tweets is C[0][1], and false Democrat tweets C[1][1]

In [ ]:
pred_logreg_BagOfWords = logistic_reg_BagOfWords.predict(X_test_BagOfWords)
confusionBoW = confusion_matrix(y_test, pred_logreg_BagOfWords)
print("Bag of Words:\n", confusionBoW)

pred_logreg_N_Gram = logistic_reg_N_Gram.predict(X_test_N_Gram)
confusionNG = confusion_matrix(y_test, pred_logreg_N_Gram)
print("N-Gram:\n", confusionNG)

Bag of Words:
 [[1993   43]
 [ 119  780]]
N-Gram:
 [[1999   37]
 [  94  805]]


#Sentiment Analysis using Convolution Neural Networks



https://towardsdatascience.com/cnn-sentiment-analysis-1d16b7c5a0e7 


In [ ]:
#data cleaning 
import re
import string

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct
df_all['Clean_Tweet'] = df_all['Tweet'].apply(lambda x: remove_punct(x))

#we tokenize the comments by using NLTK’s word_tokenize. e.g, "hello world" -> ["hello","world"]
import nltk
nltk.download('punkt')
from nltk import word_tokenize

tokens = [word_tokenize(sen) for sen in df_all.Clean_Tweet]

#set to lower case
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

#remove stopwords. Filter out certain words
nltk.download('stopwords')
stoplist = stopwords.words('english') #words to filter out

def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [removeStopWords(sen) for sen in lower_tokens]

df_all['Tweet_Final'] = [' '.join(sen) for sen in filtered_words] #sets the final text in df
df_all['tokens'] = filtered_words #sets the final tokens in the final

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
pos = []
neg = []
for l in df_all.Party: #define as positive or negative (label for supervised training)
    if l == "Democrat":
        pos.append(1)
        neg.append(0)
    elif l == "Republican":
        pos.append(0)
        neg.append(1)
df_all['Pos']= pos
df_all['Neg']= neg

df_all = df_all[['Tweet_Final', 'tokens', 'Party', 'Pos', 'Neg']]
df_all.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Tweet_Final,tokens,Party,Pos,Neg
3582,thank ddlovato describing experience dealing m...,"[thank, ddlovato, describing, experience, deal...",Democrat,1,0
49,forced move rally tonight fayetteville north c...,"[forced, move, rally, tonight, fayetteville, n...",Republican,0,1
106,’ always believed ’ best ’ one america come to...,"[’, always, believed, ’, best, ’, one, america...",Democrat,1,0
1370,donald trump responsible covid19 bear full res...,"[donald, trump, responsible, covid19, bear, fu...",Democrat,1,0
1803,unlike current president always choose science...,"[unlike, current, president, always, choose, s...",Democrat,1,0


In [ ]:
# Splits dataset into Training data and Test data

data_train, data_test = train_test_split(df_all, 
                                         test_size=0.10, 
                                         random_state=42)


In [ ]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens] #all words used in training data
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]] #finds max lenght of training data
TRAINING_VOCAB = sorted(list(set(all_training_words))) #vocabulary of words in training data
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

#do the same for the test data
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

115085 words total, with a vocabulary size of 18081
Max sentence length is 46
13066 words total, with a vocabulary size of 4529
Max sentence length is 44


In [ ]:
#creating a word2vec using reddit world news dataset https://www.kaggle.com/rootuser/worldnews-on-reddit
#what is word2vec? https://code.google.com/archive/p/word2vec/ , transform word to vector
import gensim.models
from gensim.models import Word2Vec, KeyedVectors


In [ ]:
import io
reddit_df = pd.read_csv('reddit_worldnews_start_to_2016-11-22.csv')
# Dataset is now stored in a Pandas Dataframe
reddit_df.head(10)

newsTitles  = reddit_df['title'].values
newsVec = [nltk.word_tokenize(title) for title in newsTitles]

In [ ]:
word2vec_dim = 300

In [ ]:

word2vec = Word2Vec(newsVec,min_count=1,size= word2vec_dim) #creating the word2vec model


In [ ]:
#testing the word2vec
word2vec.most_similar(['McKinsey'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Handelsblatt', 0.7613131403923035),
 ('Trends', 0.7517812252044678),
 ('Yeast', 0.7414613962173462),
 ('Inverted', 0.7381319403648376),
 ('Trend', 0.7376179695129395),
 ('YouTubers', 0.736943244934082),
 ('Mapped', 0.7368432283401489),
 ('Hotspots', 0.7347134351730347),
 ('Rankings', 0.7344036102294922),
 ('Krispy', 0.7331280708312988)]

In [ ]:
#word2vec functions
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=word2vec_dim):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train['Tweet_Final'].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train['Tweet_Final'].tolist())
train_word_index = tokenizer.word_index
testing_sequences = tokenizer.texts_to_sequences(data_test['Tweet_Final'].tolist())
test_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

MAX_SEQUENCE_LENGTH = 50
train_cnn_data = pad_sequences(training_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)
test_cnn_data = pad_sequences(testing_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)

Found 18081 unique tokens.


In [ ]:
EMBEDDING_DIM = word2vec_dim
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:

train_embedding_weights = np.zeros((len(train_word_index)+1, 
 EMBEDDING_DIM))
for word,index in train_word_index.items():
 train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(18082, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
#defining a CNN

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
 
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    convs = []
    filter_sizes = [2,3,4,5,6]
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, 
                        kernel_size=filter_size, 
                        activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)
    l_merge = concatenate(convs, axis=1)
    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [ ]:
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from gensim import models
from keras.models import Model

label_names = ['Pos', 'Neg']
model = ConvNet(train_embedding_weights, 
                MAX_SEQUENCE_LENGTH, 
                len(train_word_index)+1, 
                EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 50, 300)      5424600     input_13[0][0]                   
__________________________________________________________________________________________________
conv1d_45 (Conv1D)              (None, 49, 200)      120200      embedding_12[0][0]               
__________________________________________________________________________________________________
conv1d_46 (Conv1D)              (None, 48, 200)      180200      embedding_12[0][0]               
______________________________________________________________________________________

In [ ]:
#training the CNN
y_train = data_train[label_names].values
x_train = train_cnn_data

num_epochs = 3 #number of times data is passed through
batch_size = 200#number of data points passed through at once
hist = model.fit(x_train, 
                 y_train, 
                 epochs=num_epochs, 
                 validation_split=0.1, 
                 shuffle=True, 
                 batch_size=batch_size)

Epoch 1/3
41/41 [==============================] - 38s 916ms/step - loss: 0.1725 - acc: 0.9390 - val_loss: 0.2498 - val_acc: 0.8969
Epoch 2/3
41/41 [==============================] - 38s 918ms/step - loss: 0.1148 - acc: 0.9616 - val_loss: 0.2752 - val_acc: 0.8890
Epoch 3/3
41/41 [==============================] - 38s 918ms/step - loss: 0.0941 - acc: 0.9680 - val_loss: 0.2589 - val_acc: 0.9002


In [ ]:
#make prediction on test data using trained CNN
predictions = model.predict(test_cnn_data, 
                            batch_size=300, 
                            verbose=1)
labels = ["Democrat", "Republican"]
prediction_labels=[]

for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

print(len(predictions))
print(data_test.Party.value_counts())
sum(data_test.Party==prediction_labels)/len(prediction_labels)

4/4 [==============================] - 1s 330ms/step
992
Democrat      686
Republican    306
Name: Party, dtype: int64


0.8981854838709677